# Les blocs et le scope

---

> À l'instar de nombreux autres langages de programmation, Les `blocs` débutent par un bracket ouvrant `{` et terminent par un bracket fermant `}`, ils sont partie integrante de la syntaxe et de la logique de Rust.

> L'existance des blocs induit la notion de `scope`. Le "scope" en Rust se réfère à la portée d'une variable, définissant ainsi les parties du code où la variable est valide et accessible, ce qui permet d'assurer une gestion précise de la durée de vie des objets.

## Les blocs nécessaires

### Le bloc des fonctions

- Un des blocs les plus familiers aux programmeurs est celui des `fonctions`. Ils sont nécessaires pour chaque définition de fonction :

In [42]:
fn main() { // bracket ouvrant
    println!("Hello World !");
} // bracket fermant

> Toute définition d'une fonction sera contenue dans un bloc de code `fn ma_function(parameters...) -> return value { instructions... }`

- Cependant, si l'on se contente de déclarer qu'une fonction existe (FFI par exemple), on ne doit pas les mettre :

In [2]:
{
    extern "C" {
        // Le linker se chargera d'associer une adresse a cette fonction.
        pub fn get_inner_len(t1: *mut ::std::os::raw::c_void) -> ::std::os::raw::c_int; // pas de bloc
    }
}

()

### Le bloc des méthodes

- Tout comme les fonctions, les `méthodes` de nos types seront implémentées dans un bloc :

In [16]:
{
    struct S {}
    impl S {
        fn do_nothing(&self) {
            // bloc de la methode
        }
    }
}

()

### Le bloc des closures

- Nous le verrons plus tard, mais pour les `closures`, un bloc est nécessaire seulement si la closure contient plus d'une seule instruction :

In [9]:
{
    let closure_1 = |param: u32| {
        println!("Do something");
        42
    };
    let closure_2 = |param: u32| 42; // bloc inutile
}

()

### Le bloc des implementations

- Chaque implementation (avec le mot-clef `impl`) nécessitera un bloc ;

In [15]:
{
    struct S {}
    impl S {
        // bloc d'implementation
    }
    
    impl Drop for S {
        // bloc d'implementation
        fn drop(&mut self) {
            // bloc de methode ici
        }
    }
    unsafe impl Send for S {}
}

()

### Le bloc des définitions de type

- La définition de type `structure` ou `enumeration` nécessitent aussi des blocs, sauf tuple struct et marqueur (qui necessitent un point virgule `;` a la fin en contrepartie) :

In [48]:
{
    struct S {
        // bloc d'implementation
    }
    enum E {
        // bloc d'implementation
    }
    
    struct T(u8, u8); // Pas de bloc mais necessite un point virgule a la fin en contrepartie
    struct U; // pas de bloc mais necessite un point virgule a la fin en contrepartie
}

()

### Le bloc des boucles

- En rust, les blocs sont obligatoires pour les boucles que sont `while`, `loop`  et `for` :

In [41]:
{
    let mut i = 42;
    while i != 0 { // bloc
        i -= 1;
    }
    loop { // bloc
        break;
    }
    for i in 0..3 {
        // bloc
    }
}

()

### Le bloc des conditions

- Tout comme les blocs des boucles, il ne peut y avoir en rust de `if`, `else` sans ouverture de bracket :

In [14]:
{
    let i = 4;
    if i == 3 {
        // bloc
    } else if i == 2 {
        // bloc
    } else {
        // bloc
    }
    
    // if i == true                  // Erreur de syntaxe
    //    println!("Bananes");
}

()

### Le bloc de `if let` et de `while let` 

- Comme il s'agit de condition et de boucles, les mêmes règles s'y appliqueront, l'obligation d'ouvrir un bloc :

In [19]:
{
    let my_opt: Option<u32> = Some(42);
    if let Some(value) = my_opt {
        // bloc
    }
    let my_array = [1, 2, 3];
    let mut my_array_iterator = my_array.iter();
    while let Some(elem) = my_array_iterator.next() {
        // bloc
    }
}

()

### Le bloc du `match`

- D'abord, un match ouvre obligatoirement un bloc. Ensuite, comme pour les closures, les sous-blocs ne sont obligatoires que pour les sections contenant plus d'une seule instruction :

In [44]:
{
    let fruit = "bananes";
    let calorie = match fruit { // je dois ouvrir un bloc ici !
        "carotte" => 300, // pas besoin de bloc
        "poire" => 150, // pas besoin de bloc
        "tomate" => {
            println!("Tu aimes les tomates");
            400 // deux lignes donc le bloc est necessaire
        }
        _defaut => {
            println!("je ne connais pas ce fruit ...");
            println!("... mais aimes-tu vraiment les {} au moins ?", _defaut);
            0  // retour du cas par defaut
        }
    };
}

je ne connais pas ce fruit ...
... mais aimes-tu vraiment les bananes au moins ?


()

### Le bloc des modules / namespaces

- On ne peut **définir** un `module` sans ouvrir de bloc :

In [43]:
{
    mod test {
        // bloc du module
    }
}

()

> *Il existe d'autres types de bloc dont je ne parlerai pas ici.*

## Les blocs et durée de vie

- Il arrive que l'on déclare un bloc afin d'être certain que les fonctions, les variables, etc .... définies à l'intérieur de ce dernier ne seront plus accessibles depuis l'extérieur de ce bloc. Ce bloc constitue un `scope` :

In [45]:
fn main() {
    { // j'ouvre un bloc
        let s: String = "ma_string".into();
        dbg!(&s);
    }
    // ma variable s n'existe plus ici, elle est `out of scope`
    println!("Hello world!");
}
main();

[src/lib.rs:11] &s = "ma_string"


Hello world!


> C'est d'ailleurs pour cette raison que je mets ici, sur Jupyter, les sections de code dans des blocs.

In [47]:
fn exemple() {
    // ici debute le scope de la fonction exemple
    let s: String = "banane".into(); // ma variable s vivra dans tout le bloc de la fonction ...
                                     // ... sauf en cas de "move". (mais on en fera pas dans cet exemple) 
    {
        // j'ai ouvert un nouveau bloc
        dbg!(&s); // A l'interieur de mon sous-bloc, j'ai toujours acces a s qui vient du scope parent
        let i = 42; // La variable i ici ne vivra que dans ce sous-bloc et ses enfants
        {
            // Encore un sous bloc
            dbg!(&i); // i est accessible
        }
        dbg!(&i); // i est accessible ici aussi
    }
    // la variable i du sous-bloc precedent n'est plus dans le scope. je ne peux pas la lire ici.
    dbg!(&s); // Par contre, ma string s est toujouts valide puisque declaree au debut de ce bloc.
}
exemple();
// Sans surprise, je n'ai acces ici ni a la string s et encore moins a la variable i !
// De plus, une fonction ne peut pas capturer son environnement.

[src/lib.rs:14] &s = "banane"
[src/lib.rs:18] &i = 42
[src/lib.rs:20] &i = 42
[src/lib.rs:23] &s = "banane"


## Le bloc d'assignation

- Enfin. Il existe la possibilité d'ouvrir un bloc pour assigner des variables avec plusieurs instructions:

In [40]:
{
    let ma_var = { // Bloc d'assignation
        println!("Woot ?"); // une instruction
        42 // le retour de mon bloc
    };
    dbg!(ma_var);
}

Woot ?


[src/lib.rs:142] ma_var = 42


()

**NB : NOUS POUVONS CONSTATER AVEC LES EXEMPLES PRÉSENTES ICI QU'UN BLOC PEUT RETOURNER QUELQUE CHOSE, ON DIT QUE CE BLOC EST UNE EXPRESSION. NOUS VERRONS CELA PLUS EN DÉTAIL LORS DE LA PRÉSENTATION DES CONCEPTS CLEF DE RUST.**